In [1]:
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using JLD2
using Statistics
using MLJBase
using BSON: @load # for load weights
using Plots
using Zygote
using Distributions

In [9]:
train_data = JLD2.load_object("data2/train_data.jld2");
test_data = JLD2.load_object("data2/test_data.jld2");
maxCycle = 50 # maximum number of cycles
inputSize = 11
outputSize = 6
tryNumber = 1

1

In [17]:
train_data = JLD2.load_object("data/train_data.jld2");
maxCycle = 150 # maximum number of cycles
inputSize = 19
outputSize = 2

2

In [18]:
model1 = Chain(Dense(inputSize, 2*inputSize, sigmoid), Dense(2*inputSize, outputSize))#define the model 

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 3.523 KiB.

In [10]:

function initABC()
    for i in 1:pop
        for j in 1:Dimension
            temp = lowerBound + rand()*preCal
            if temp < lowerBound
                temp = lowerBound
            elseif temp > upperBound
                temp = upperBound
            end
            globalWeights[i][j] = temp
        end
        globalFoodSourceFitness[i] = calculateFitness(globalWeights[i])
        
        globalfoodSourceCount[i] = 0
    end
    calCumFitness();
    bestWeight = globalWeights[getBest()];
end

function FindNewFoodSource(index)
    j = index
    while j == index
        j = rand(1:pop)
    end
    newSource = zeros(Float64,Dimension)
    for i in 1:Dimension
        
        temp = globalWeights[index][i] + acc*rand(Uniform(-1,1))*(globalWeights[index][i]-globalWeights[j][i])
        if temp < lowerBound
            temp = lowerBound
        elseif temp > upperBound
            temp = upperBound
        end
        newSource[i] = temp
    end
    updateFoodSource(index, newSource)
end


function updateFoodSource(index, newSource)
    localFitness = calculateFitness(newSource)
    if localFitness > globalFoodSourceFitness[index]
        globalFoodSourceFitness[index] = localFitness
        globalWeights[index] = newSource
        globalfoodSourceCount[index] = 0
    else
        globalfoodSourceCount[index] = globalfoodSourceCount[index]+1
    end
    calCumFitness();
end


function calculateFitness(newWeight)
    return 1/(1+loss(train_data, re(newWeight)))
end



function checkRetire()
    for i in 1:pop
        if globalfoodSourceCount[i] > tryLimit
            if i == getBest()
                globalfoodSourceCount[i] = 0
            else
            FindNewFoodSource(i)
            end
        end
    end
end

function calCumFitness()
    fitnessSum = sum(globalFoodSourceFitness)
    fitnessProb = globalFoodSourceFitness/fitnessSum
    temp =  cumsum(fitnessProb);
    for i in 1: pop
        cumProb[i] = temp[i];
    end
end

function rouletteWheelSelection()
    randNum = rand(Uniform(0,1))
    for i in 1:pop
        if randNum <= cumProb[i]
            return i
        end
    end
end



function getBest()
    return argmax(globalFoodSourceFitness)
end


function employedBeeFindNew()
    for i in 1:pop
        FindNewFoodSource(i)
    end
end


function onlookFindNew()
    for i in 1:pop
        index = rouletteWheelSelection()
        FindNewFoodSource(index)
    end
end


function scoutBeeFindNew()
    checkRetire()
end

accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)

function swapBest(index,gbestWeight)
    if accuracy(train_data, re(bestWeight)) < accuracy(train_data, re(globalWeights[index]))
        return globalWeights[index];
    else
        return bestWeight;
    end
end

swapBest (generic function with 1 method)

In [11]:
accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)

loss(model) = (x,y)->Flux.logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))


loss (generic function with 3 methods)

ABC

In [12]:
model = Chain(Dense(inputSize, 2*inputSize, sigmoid), Dense(2*inputSize, outputSize)) #define the model 
θ, re = Flux.destructure(model); 
Dimension = size(θ)[1]; #number of parameters

pop = 100 # number of food sources
tryLimit = 70 # limit of the number of tries
acc = 1 # acceleration constant

lowerBound = -3
upperBound = 2

rng =Random.default_rng()

globalWeights = [Vector{Float64}(undef,Dimension) for _ in 1:pop]; 
    globalFoodSourceFitness = zeros(Float64, pop)
    cumProb = zeros(Float64, pop)
    globalfoodSourceCount = zeros(Float64, pop)
    bestWeight = Vector{Float64}(undef,Dimension);

preCal = upperBound-lowerBound;



In [13]:
ABCsave_resultTrain = [Vector{Float64}(undef,maxCycle) for _ in 1:tryNumber];
ABCsave_resultTest = [Vector{Float64}(undef,maxCycle) for _ in 1:tryNumber];
for i in 1:tryNumber
    ABCsave_resultTrain[i] = zeros(Float64,maxCycle);
    ABCsave_resultTest[i] = zeros(Float64,maxCycle);
end

In [15]:
for k in 1:tryNumber
    globalWeights = [Vector{Float64}(undef,Dimension) for _ in 1:pop]; 
    globalFoodSourceFitness = zeros(Float64, pop)
    cumProb = zeros(Float64, pop)
    globalfoodSourceCount = zeros(Float64, pop)
    bestWeight = Vector{Float64}(undef,Dimension);
    initABC()
    for i in 1:maxCycle
        employedBeeFindNew()
        onlookFindNew()
        scoutBeeFindNew()
        # bestIndex = getBest()
        bestWeight = swapBest(getBest(), bestWeight);    #print("Cycle: ", i, "Best Fitness: ", globalFoodSourceFitness[bestIndex])
        ABCsave_resultTrain[k][i] = ABCsave_resultTrain[k][i] + accuracy(train_data, re(bestWeight))
        ABCsave_resultTest[k][i] = ABCsave_resultTest[k][i] + accuracy(test_data, re(bestWeight))
        #print("\n")
        
    end
    JLD2.save_object("ABCsave_resultTrain.jld2", ABCsave_resultTrain)
    JLD2.save_object("ABCsave_resultTest.jld2", ABCsave_resultTest)
end

In [ ]:
# plot!((1:maxCycle), ABCsave_resultTrain[k],label = "ABC", title = "Accuracy", legend = :outertopleft)



In [ ]:
plot!((1:maxCycle), ABCsave_resultTrain[2],label = "BP", title = "Accuracy", legend = :outertopleft)

BP

In [5]:
# change population_size will increase training time, but may increase the accuracy
BPsave_resultTrain = zeros(maxCycle);
BPsave_resultTest = zeros(maxCycle);
# l1 = Dense(19, 2*19, sigmoid)
# l2 = Dense(2*19, 2)
model2 = Chain(Dense(inputSize, 2*inputSize, sigmoid), Dense(2*inputSize, outputSize))#define the model 


for j in 1:maxCycle
    Flux.train!(loss(model2), Flux.params(model2), train_data, ADAM(1e-4))
    BPsave_resultTrain[j] = BPsave_resultTrain[j] + accuracy(train_data, model2)
    BPsave_resultTest[j] = BPsave_resultTest[j] + accuracy(test_data, model2)
end

JLD2.save_object("BPsave_resultTrain.jld2", BPsave_resultTrain)
JLD2.save_object("BPsave_resultTest.jld2", BPsave_resultTest)

In [ ]:
plot((1:maxCycle), ABCsave_resultTrain,label="ABC")
plot!((1:maxCycle), BPsave_resultTrain,label = "BP", title = "Accuracy", legend = :outertopleft)

PSO

In [6]:
maxCycle = 5000

5000

In [7]:
N = 50
w0 = 0.8
c1 = 1.6
c2 = 1.8

1.8

In [16]:
model1 = Chain(Dense(inputSize, 2*inputSize, sigmoid), Dense(2*inputSize, outputSize))#define the model 

Chain(
  Dense(11, 22, σ),                     # 264 parameters
  Dense(22, 6),                         # 138 parameters
)                   # Total: 4 arrays, 402 parameters, 1.820 KiB.

In [8]:
# change population_size will increase training time, but may increase the accuracy
PSOsave_resultTrain = zeros(maxCycle);
PSOsave_resultTest = zeros(maxCycle);
# l1 = Dense(19, 2*19, sigmoid)
# l2 = Dense(2*19, 2)
model1 = Chain(Dense(inputSize, 2*inputSize, sigmoid), Dense(2*inputSize, outputSize))#define the model 
θ, re = Flux.destructure(model1)
weights = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:N];
vec = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:N];

for i in 1:N
    vec[i] = rand(size(θ)[1])
    for j in 1:size(θ)[1]
        weights[i][j] = rand(Normal(θ[i], 0.05))
    end

end
p_best = copy(weights);
g_best = sort(weights, lt=(x,y)->isless(loss(train_data, re(x)), loss(train_data, re(y))))[1];

for i in 1:maxCycle
    for j in 1:N
        vec[j] = w0 *vec[j] + c1 *(1.0 - rand()) * (p_best[j] - vec[j]) + c2 * (1.0 - rand()) * (g_best - vec[j])
        weights[j] = vec[j]+weights[j]
        if loss(train_data, re(weights[j])) < loss(train_data, re(p_best[j]))
            p_best[j] = weights[j]
        end
    end
    g_best = sort(p_best, lt=(x,y)->isless(loss(train_data, re(x)), loss(train_data, re(y))))[1];

    PSOsave_resultTrain[i] = PSOsave_resultTrain[i] + accuracy(train_data, re(g_best));
    PSOsave_resultTest[i] = PSOsave_resultTest[i] + accuracy(test_data, re(g_best));
    JLD2.save_object("PSOsave_resultTrain.jld2", PSOsave_resultTrain)
    JLD2.save_object("PSOsave_resultTest.jld2", PSOsave_resultTest)

end



In [ ]:
plot((1:maxCycle), ABCsave_resultTrain,label="ABC")
plot!((1:maxCycle), PSOsave_resultTrain,label = "PSO", title = "Accuracy", legend = :outertopleft)